# 1) Imports

In [1]:
# # http://pytorch.org/
# from os.path import exists
# from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
# platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
# cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
# accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

# !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
# import torch
!pip install torch torchvision

    100% |████████████████████████████████| 591.8MB 26kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x60e64000 @  0x7f24b50e52a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 21.8MB/s 
    100% |████████████████████████████████| 2.0MB 4.1MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms 
from torch.autograd import Variable 

#2) Datasets

In [3]:
train_dataset=dsets.MNIST(root='./data',train=True,transform=transforms.ToTensor(),download=True)
test_dataset=dsets.MNIST(root='./data',train=False,transform=transforms.ToTensor(),download=True)

Processing...
Done!


In [0]:
#hyperparameters

batch_size=100
iters=3000
learning_rate=0.01
num_epochs=int(iters*batch_size/len(train_dataset))

In [5]:
print(num_epochs)

5


#3) Making the dataset iterable

In [0]:
train_loader =  torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader  =  torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)


#4) Making the class model

In [0]:
class CNNmodel(nn.Module):
  def __init__(self):
    super(CNNmodel,self).__init__()
    
    #Conv layer 1
    
    self.conv1=nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=2) #out-size 28
    self.relu1=nn.ReLU()
    #MaxPooling layer 1
    
    self.maxpool1=nn.MaxPool2d(kernel_size=2) #out-size 14
    
    #Conv layer 2
    
    self.conv2=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=2) #out-size 14
    self.relu2=nn.ReLU()
    
    #MaxPooling layer 2
    
    self.maxpool2=nn.MaxPool2d(kernel_size=2) #out-size 7
    
    #Readout layer
    # 1 x 32 x 7 x 7
    self.linear=nn.Linear(32*7*7,10)    #nn.Linear takes (input_dim,output_dim)
    
  def forward(self,x):
    # 1
    out=self.conv1(x)
    
    out=self.relu1(out)
    
    out=self.maxpool1(out)
    
    #2
    out=self.conv2(out)
    
    out=self.relu2(out)
    
    out=self.maxpool2(out)
    
    out=out.view(out.size(0),-1)
    
    out = self.linear(out)
    
    return out

#5) Instantiating a model class

In [0]:
model=CNNmodel()


In [0]:
#selecting a loss class
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

#6) Training model class

In [10]:
iters=0
for epochs in range(num_epochs):
  for i,(images,labels) in enumerate(train_loader):
    
    #converting images and labels to variables so as to calculate gradient 
    images=Variable(images)
    labels=Variable(labels)
  
    #clearing gradient buffers
    optimizer.zero_grad()
    
    #finding outputs
    output=model(images)
    
    #calculating losses
    loss=criterion(output,labels)
  
    #backpropagating the loss
    loss.backward()
    
    #updating the parameters
    optimizer.step()
    
    #updating counter
    iters+=1
    
    if iters%500==0:
      correct=0
      total=0
      for images,labels in test_loader:
        
        images=Variable(images)
        
        labels=Variable(labels)
        
        output=model(images)
        
        _,predicted=torch.max(output.data,1)             # added .data after output 
        
        correct += (predicted == labels).sum().item()
        
        total+=labels.size(0)
        
        accuracy=(100*correct)/total
      print('Iteration:{}    Loss:{}    Accuracy:{}' .format(iters,loss,accuracy) )
    

Iteration:500    Loss:0.4428042471408844    Accuracy:89.51
Iteration:1000    Loss:0.28307169675827026    Accuracy:93.34
Iteration:1500    Loss:0.16203220188617706    Accuracy:94.81
Iteration:2000    Loss:0.1406887024641037    Accuracy:95.84
Iteration:2500    Loss:0.07750476151704788    Accuracy:96.49
Iteration:3000    Loss:0.18041855096817017    Accuracy:96.79
